In [30]:
#Relax Inc.

In [31]:
#Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate days  in  at  least  
#one  seven­day  period ,  identify  which  factors  predict  future  user adoption .
#We  suggest  spending  1­2  hours  on  this,  but  you're  welcome  to  spend  more  or  less.
#Please  send  us  a  brief  writeup  of  your  findings  (the  more  concise,  the  better  ­­  no  more
#than  one  page),  along  with  any  summary  tables,  graphs,  code,  or  queries  that  can  help
#us  understand  your  approach.  Please  note  any  factors  you  considered  or  investigation you  did,  even  if  they  
#did  not  pan  out.  Feel  free  to  identify  any  further  research  or  data you  think  would  be  valuable

In [88]:
#Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import datetime
pd.options.display.max_columns = 500
%matplotlib inline

In [89]:
df1= pd.read_csv('takehome_user.csv')
df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [90]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [112]:
df2=pd.read_csv('relax_challenge/takehome_user_engagement.csv')
df2

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [113]:
#convert time_stamp column to a datetime object
time_stamp = pd.to_datetime(df2['time_stamp'])

In [114]:
time_stamp.sort_values(ascending=True)

178140   2012-05-31 08:20:06
59486    2012-05-31 15:47:36
175638   2012-05-31 17:19:37
26821    2012-05-31 21:58:33
109716   2012-06-01 00:17:30
                 ...        
84316    2014-06-04 23:32:13
162633   2014-06-04 23:34:04
168409   2014-06-04 23:46:31
6053     2014-06-04 23:56:26
70763    2014-06-06 14:58:50
Name: time_stamp, Length: 207917, dtype: datetime64[ns]

In [115]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [116]:
# convert from string to datetime format
df2['time_stamp'] = pd.to_datetime(df2['time_stamp'])
# set the time_stamp column as the dataframe index
df2 = df2.set_index('time_stamp')
df2

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1
...,...,...
2013-09-06 06:14:15,11996,1
2013-01-15 18:28:37,11997,1
2014-04-27 12:45:16,11998,1


In [117]:
# group data by user_id, resample into weekly dates, and sum the visited column
df2 = df2.groupby('user_id').resample('1W').sum()
df2

user_id  visited
user_id time_stamp                  
1       2014-04-27        1        1
2       2013-11-17        2        1
        2013-11-24        0        0
        2013-12-01        2        1
        2013-12-08        0        0
...                     ...      ...
11996   2013-09-08    11996        1
11997   2013-01-20    11997        1
11998   2014-04-27    11998        1
11999   2012-06-03    11999        1
12000   2014-01-26    12000        1

[82282 rows x 2 columns]

In [119]:
# create dataframe of only the adopted users
adopted = df2[df2['visited'] > 2]
del adopted['user_id']
adopted

visited
user_id time_stamp         
2       2014-02-09        3
10      2013-03-03        3
        2013-04-14        3
        2013-04-28        3
        2013-05-05        4
...                     ...
11988   2014-04-06        3
        2014-05-04        4
        2014-05-18        3
        2014-05-25        5
        2014-06-01        3

[33859 rows x 1 columns]

In [120]:
# create user id list for adopted users
adopted_index = list(adopted.index)
adopted_users_list = []
for i in range(len(adopted_index)):
    adopted_users_list.append(adopted_index[i][0])
len(adopted_users_list)

33859

In [124]:
# remove duplicate user ids
adopted_users = set(adopted_users_list)
len(adopted_users)

1445

In [125]:
# add an adopted column to the dataframe with a default value of False
df1['adopted'] = False

# use the adopted_users set to label the adopted users in the dataframe
for user in adopted_users:
    df1.loc[user, 'adopted'] = True

df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,False
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,True
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
